Filtrado colaborativo
Los Sistemas Recomendadores son una colección de algoritmos utilizados para sugerir temas a los usuarios, basados en información tomada desde el punto de vista del usuario. Estos sistemas ubicuos pueden ser comúnmente vistos en tiendas online, bases de datos de películas y buscadores de empleos. Vamos a ver los sistemas de recomendación basados en Contenido e implementaremos una versión utilizando Python y la librería Pandas.

Primero, importemos las librerías necesarias:

In [ ]:
#librería de manipulación de dataframes
import pandas as pd
#Funciones matemáticas, necesitaremos sólo importar la función sqrt
from math import sqrt
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

Adquisición de Datos
Para adquirir y extraer los datos, simplemente ejecuta los siguientes scripts Bash:

In [ ]:
	!wget -O moviedataset.zip https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0101ENv3/labs/moviedataset.zip
print('unziping ...')
!unzip -o -j moviedataset.zip

Ahora carguemos cada archivo dentro de su dataframe:

In [ ]:
#Guardando la información de la película dentro de un dataframe de panda
movies_df = pd.read_csv('movies.csv')
#Guardando información del usuario dentro de un dataframe de panda
ratings_df = pd.read_csv('ratings.csv')

Miremos cada uno de ellos a ver cómo están organizados:

In [ ]:
#Head es una función que obtiene las primeras N filas de un dataframe. El valor por omisión de N es 5.
movies_df.head()

Cada película tiene un único ID, un título con su año de estreno (El cual puede contener caracteres unicode) y muchos géneros diferentes en el mismo campo. Saquemos el año de la columna del título y ubiquemoslo en su propia columna con la función extract que viene con Pandas.
Saquemos el año de la columna title utilizando la función de pandas replace y guardémoslo en la nueva columna year.

In [ ]:
#Utilizar expresiones regulares para encontrar un año guardado entre paréntesis
#Especificamos los paréntesis de forma tal de que no haya problemas con las películas que tiene el año en sus títulos
movies_df['year'] = movies_df.title.str.extract('(<img alt="\d\d\d\d" title="Rendered by QuickLaTeX.com" height="2" width="3" style="vertical-align: -4px;" data-src="https://www.statdeveloper.com/wp-content/ql-cache/quicklatex.com-6bed55770ce78fa33f73530474cdaed1_l3.png" class="ql-img-inline-formula quicklatex-auto-format lazyload" src="data:image/gif;base64,R0lGODlhAQABAAAAACH5BAEKAAEALAAAAAABAAEAAAICTAEAOw=="><noscript><img   alt="\d\d\d\d" title="Rendered by QuickLaTeX.com" height="2" width="3" style="vertical-align: -4px;" data-src="https://www.statdeveloper.com/wp-content/ql-cache/quicklatex.com-6bed55770ce78fa33f73530474cdaed1_l3.png" class="ql-img-inline-formula quicklatex-auto-format lazyload" src="data:image/gif;base64,R0lGODlhAQABAAAAACH5BAEKAAEALAAAAAABAAEAAAICTAEAOw==" /><noscript><img src="https://www.statdeveloper.com/wp-content/ql-cache/quicklatex.com-6bed55770ce78fa33f73530474cdaed1_l3.png" class="ql-img-inline-formula quicklatex-auto-format" alt="\d\d\d\d" title="Rendered by QuickLaTeX.com" height="2" width="3" style="vertical-align: -4px;"/></noscript>)',expand=False)
#Sacando los paréntesis
movies_df['year'] = movies_df.year.str.extract('(\d\d\d\d)',expand=False)
#Sacando los años de la columna 'title'
movies_df['title'] = movies_df.title.str.replace('(<img alt="\d\d\d\d" title="Rendered by QuickLaTeX.com" height="2" width="3" style="vertical-align: -4px;" data-src="https://www.statdeveloper.com/wp-content/ql-cache/quicklatex.com-6bed55770ce78fa33f73530474cdaed1_l3.png" class="ql-img-inline-formula quicklatex-auto-format lazyload" src="data:image/gif;base64,R0lGODlhAQABAAAAACH5BAEKAAEALAAAAAABAAEAAAICTAEAOw=="><noscript><img   alt="\d\d\d\d" title="Rendered by QuickLaTeX.com" height="2" width="3" style="vertical-align: -4px;" data-src="https://www.statdeveloper.com/wp-content/ql-cache/quicklatex.com-6bed55770ce78fa33f73530474cdaed1_l3.png" class="ql-img-inline-formula quicklatex-auto-format lazyload" src="data:image/gif;base64,R0lGODlhAQABAAAAACH5BAEKAAEALAAAAAABAAEAAAICTAEAOw==" /><noscript><img src="https://www.statdeveloper.com/wp-content/ql-cache/quicklatex.com-6bed55770ce78fa33f73530474cdaed1_l3.png" class="ql-img-inline-formula quicklatex-auto-format" alt="\d\d\d\d" title="Rendered by QuickLaTeX.com" height="2" width="3" style="vertical-align: -4px;"/></noscript>)', '')
#Aplicando la función strip para sacar los espacios finales que pudiera haber
movies_df['title'] = movies_df['title'].apply(lambda x: x.strip()

Veamos el resultado

In [ ]:
movies_df.head()

Luego, saquemos la columna de los géneros ya que no los necesitaremos para este sistema recomendador.

In [ ]:
#Eliminando la columna géneros
movies_df = movies_df.drop('genres', 1)

Aquí está el dataframe final:

In [ ]:
movies_df.head()

Ahora, veamos el dataframe de los ratings.

In [ ]:
ratings_df.head()

Cada fila en el dataframe ratings tiene un id de usuario asociado con al menos una película, un rating y una marca de tiempo que muestra cuando se revisó. No se necesitará la columna timestamp, por lo que se eliminará para ahorrar memoria.

In [ ]:
#Drop elimina una fila en particular o columna dentro de un dataframe
ratings_df = ratings_df.drop('timestamp', 1)

Asi es cómo el Dataframe definitivo se ve:

In [ ]:
ratings_df.head()

Filtrado Colaborativo
Ahora, comenzaremos el trabajo con los sistemas recomendadores.

La primer técnica que vas a ver se llama Filtrado Colaborativo, también conocido como Filtrado de Usuario a Usuario. Como lo indica su nombre alternativo, esta técnica utiliza otros usuarios para recomendar items al ingreso de datos. Se intenta encontrar usuarios que tengan preferencias y opiniones parecidas para entonces recomendar items que se hayan parecido al ingreso anterior. Existen varios métodos para encontrar usuarios parecidos (Incluso los que utilizan Machine Learning), y el método que vamos a utilizar estará basado en la Función de Correlación de Pearson.
El proceso para crear un sistema de recomendación Basado en el Usuario es el siguiente:

Elegir un usuario con las películas que el usuario ha mirado
Basado en su indice de selección de películas, encuentra a los primeros X vecinos
Obtener el registro de la película que miró el usuario para cada vecino.
Calcular un puntaje de similitud utilizando alguna fórmula
Recomendar los ítems con los puntajes más altos
Comencemos creando un usuario a quien recomendar películas:
Nota: Para agregar más películas, aumenta la catidad de elementos en userInput. Agrega tantos como desees! Solo asegúrate de escribir en letras mayúsculas y si una película comienza con un “The”, como “The Matrix” entonces escríbelo así: ‘Matrix, The’.

In [ ]:
userInput = [
            {'title':'Breakfast Club, The', 'rating':5},
            {'title':'Toy Story', 'rating':3.5},
            {'title':'Jumanji', 'rating':2},
            {'title':"Pulp Fiction", 'rating':5},
            {'title':'Akira', 'rating':4.5}
         ] 
inputMovies = pd.DataFrame(userInput)
inputMovies

Agregar movieId al ingreso del usuario: Con las datos ingresados completos, extraigamos los ID de las películas del dataframe de películas y agreguémosla.
Esto se logra primero sacando las filas que tienen que tienen títulos de películas y luego une este subconjunto con el dataframe de entrada. También sacamos columnas que no se necesitan para ahorrar espacio de memoria.

In [ ]:
#Filtrar las películas por título
inputId = movies_df[movies_df['title'].isin(inputMovies['title'].tolist())]
#Luego juntarlas para obtener el movieId. Implícitamente, lo está uniendo por título.
inputMovies = pd.merge(inputId, inputMovies)
#Eliminando información que no utilizaremos del dataframe de entrada
inputMovies = inputMovies.drop('year', 1)
#Dataframe de entrada final
#Si una película que se agregó no se encuentra, entonces podría no estar en el dataframe 
#original o podría estar escrito de otra forma, por favor revisar mayúscula o minúscula.
inputMovies

Usuarios que han visto las mismas películas: Ahora, que el ID de la película está como entrada, podemos obtener el subconjunto de usuarios que han visto o revisado las películas en nuestra entrada.

In [ ]:
#Filtrando los usuarios que han visto las películas y guardándolas
userSubset = ratings_df[ratings_df['movieId'].isin(inputMovies['movieId'].tolist())]
userSubset.head()

Ahora agrupamos las filas por el ID del usuario.

In [ ]:
#Groupby crea varios dataframes donde todos tienen el mismo valor para la columna especificada como parámetro
userSubsetGroup = userSubset.groupby(['userId'])

Miremos a uno de los usuarios, por ejemlo, el userID=1130

In [ ]:
userSubsetGroup.get_group(1130)

Ahora hagamos ordenamiento dentro de cada grupo de forma tal que los usuarios que compartan la mayor cantidad de películas tengan prioridad. Esto brinda una recomendación mejorada ya que no será necesario pasar por todos los usuarios.

In [ ]:
#Ordenamiento de forma tal de que los usuarios con más películas en común tengan prioridad
userSubsetGroup = sorted(userSubsetGroup,  key=lambda x: len(x[1]), reverse=True)

Ahora, miremos al primer usuario

In [ ]:
userSubsetGroup[0:3]

Similitud entre usuarios y usuarios ingresantes
Luego, compararemos a todos los usuarios (casi todos) con nuestro usuario indicado y encontraremos el que más se parece.
Encontraremos cómo cada usuario similar se relacionan entre si a través del Coeficiente de Correlación de Pearson. Se utiliza para medir la fuerza de una asociación lineal entre dos variables. La fórmula para encontrar este coeficiente entre los conjuntos X e Y con los valores de N se puede ver en la fórmula:
r=\frac{\sum_{i=1}^{n}(x_i-\bar{x})(y_i-\bar{y})}{\sqrt{\sum_{i=1}^{n}(x_i-\bar{x})^2}\sqrt{\sum_{i=1}^{n}(y_i-\bar{y})^2}}
¿Por qué la Correlación Pearson?
La correlación Pearson no varía con la escala, ejemplo: si se multiplican todos los elementos por una constante distinta a cero o si se agrega cualquier constante a todos los elementos. Por ejemplo, si tienes dos vectores X e Y, entonces, pearson(X, Y)=pearson(X, 2 \cdot Y + 3). Esta es una propiedad muy importante en los sistemas recomendadores por que dos usuarios podrían putnuar dos series de elementos de manera completamente diferente, pero serían usuarios parecidos (ejemplo: con ideas similares) con puntuaciones parecidas en escalas variadas .

Los valores brindados por la fórumula puede variar de r=-1 a r=1, donde 1 se correlaciona directamente entre las dos entidades (esto sería una correlación positiva perfecta) y -1 forma una correlación negativa perfecta. En nuestro caso, un 1 se refiere a que dos usuarios tiene gustos parecidos, mientras que -1 es lo opuesto.
Elegiremos un subconjunto de usuarios para hacer las iteraciones. Este limite existe porque no queremos desperdiciar mucho tiempo pasando por cada usuario.

In [ ]:
userSubsetGroup = userSubsetGroup[0:100]

Ahora, calculemos la Correlación Pearson entre la entrada del usuario el grupo, para almacenarlo en el diccionario, donde la clave es el Id del usuario y el valor es el coeficiente

In [ ]:
#Guardar la Correlación Pearson en un diccionario, donde la clave es el Id del usuario y el valor es el coeficiente
pearsonCorrelationDict = {]
#Para cada grupo de usuarios en nuestro subconjunto 
for name, group in userSubsetGroup:
    #Comencemos ordenando el usuario actual y el ingresado de forma tal que los valores no se mezclen luego
    group = group.sort_values(by='movieId')
    inputMovies = inputMovies.sort_values(by='movieId')
    #Obtener el N para la fórmula
    nRatings = len(group)
    #Obtener los puntajes de revisión para las películas en común
    temp_df = inputMovies[inputMovies['movieId'].isin(group['movieId'].tolist())]
    #Guardarlas en una variable temporal con formato de lista para facilitar cálculos futuros
    tempRatingList = temp_df['rating'].tolist()
    #Pongamos también las revisiones de grupos de usuarios en una lista
    tempGroupList = group['rating'].tolist()
    #Calculemos la Correlación Pearson entre dos usuarios, x e y
    Sxx = sum([i**2 for i in tempRatingList]) - pow(sum(tempRatingList),2)/float(nRatings)
    Syy = sum([i**2 for i in tempGroupList]) - pow(sum(tempGroupList),2)/float(nRatings)
    Sxy = sum( i*j for i, j in zip(tempRatingList, tempGroupList)) - sum(tempRatingList)*sum(tempGroupList)/float(nRatings)
    #Si el denominador es diferente a cero, entonces dividir, sino, la correlación es 0.
    if Sxx != 0 and Syy != 0:
        pearsonCorrelationDict[name] = Sxy/sqrt(Sxx*Syy)
    else:
        pearsonCorrelationDict[name] = 0

In [ ]:
pearsonCorrelationDict.items()

In [ ]:
pearsonDF = pd.DataFrame.from_dict(pearsonCorrelationDict, orient='index')
pearsonDF.columns = ['similarityIndex']
pearsonDF['userId'] = pearsonDF.index
pearsonDF.index = range(len(pearsonDF))
pearsonDF.head()

Ahora obtengamos los 50 primeros usuarios más parecidos a los que se ingresaron.

In [ ]:
topUsers=pearsonDF.sort_values(by='similarityIndex', ascending=False)[0:50]
topUsers.head()

Recomendemos películas al usuario de entrada puntuando a los usuarios elegidos para todas las películas

Haremos esto tomando el peso promedio de los ratings de las películas utilizando la Correlación Pearson. Pero para hacer esto, primero necesitamos que los usuarios vean las películas en nuestro pearsonDF a partir del dataframe de puntajes y luego guardar su correlación en una nueva columna llamada _similarityIndex”. Estos se logra juntando estas dos tablas de debajo.

In [ ]:
topUsersRating=topUsers.merge(ratings_df, left_on='userId', right_on='userId', how='inner')
topUsersRating.head()

Ahora todo lo que se necesita hacer es multiplicar el puntaje de la película por su peso (El índice de similitud), luego se suman los nuevos puntajes y dividen por la suma de los pesos.
Esto se logra sencillamente multiplicando dos columnas, luego agrupando el dataframe por la columna movieId y luego dividiendo dos columnas:
Aqui se muestra la idea de todos los usuarios similares respecto de las películas candidatas para el usuario ingresado:

In [ ]:
#Se multiplica la similitud de los puntajes de los usuarios
topUsersRating['weightedRating'] = topUsersRating['similarityIndex']*topUsersRating['rating']
topUsersRating.head()

In [ ]:
#Se aplica una suma a los topUsers luego de agruparlos por userId
tempTopUsersRating = topUsersRating.groupby('movieId').sum()[['similarityIndex','weightedRating']]
tempTopUsersRating.columns = ['sum_similarityIndex','sum_weightedRating']
tempTopUsersRating.head()

In [ ]:
#Se crea un dataframe vacío
recommendation_df = pd.DataFrame()
#Ahora se toma el promedio ponderado
recommendation_df['weighted average recommendation score'] = tempTopUsersRating['sum_weightedRating']/tempTopUsersRating['sum_similarityIndex']
recommendation_df['movieId'] = tempTopUsersRating.index
recommendation_df.head()

Luego, ordenémoslo y veamos las primeras 20 películas que el algoritmo recomendó!

In [ ]:
recommendation_df = recommendation_df.sort_values(by='weighted average recommendation score', ascending=False)
recommendation_df.head(10)

In [ ]:
movies_df.loc[movies_df['movieId'].isin(recommendation_df.head(10)['movieId'].tolist())]

Ventajas y Desventajas del Filtro Colaborativo
Ventajas
Tiene en cuenta el puntaje de otros usuarios
No necesita estudiar o extraer la información del elemento recomendado
Se adapta al interés del usuario mientras cambia
Desventajas
La función de aproximación puede ser lenta
Podriá surgir una cantidad baja de usuarios para aproximar
Temas de privacidad cuando se intenta aprender de las preferencias del usuario